In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scs
from math import  inf

In [ ]:
from docplex.mp.model import Model
n=Model(name='farmer problem using reduced scenario')
#parameters
yields={1:[2,2.5,3] , 2:[2.4,3,3.6] , 3:[16,20,24]}
probability={1:0.3 , 2:0.4 , 3:0.3}
plant=[150,230,260]
buy=[238,210]
sell=[170, 150, 36, 10]
#variables
x=n.continuous_var_matrix(range(1,4),range(1,3),name='x',key_format='%s')
y=n.continuous_var_cube(range(1,3),range(1,4),range(2,4),name='y',key_format='%s')
w=n.continuous_var_cube(range(1,5),range(1,4),range(2,4),name='w',key_format='%s')
#constraints